In [104]:
import zipfile
import os

In [105]:
# def ex(zip_path,ex_path):
#   with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(ex_path)
#     print(os.listdir(ex_path))
# ex("/content/archive.zip","/content/")

In [106]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !cp /content/movies.csv /content/drive/MyDrive/

In [ ]:
# !cp /content/ratings.csv /content/drive/MyDrive/

In [111]:
import pandas as pd
import spacy
nlp=spacy.load("en_core_web_sm")
# movies=pd.read_csv("movies.csv")
# ratings=pd.read_csv("ratings.csv")

In [112]:
movies=pd.read_csv("/content/drive/MyDrive/movies.csv")
ratings=pd.read_csv("/content/drive/MyDrive/ratings.csv")

In [113]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [114]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [118]:
import re
from spacy.lang.en.stop_words import STOP_WORDS
def clean_title(title):
  cleaned=re.sub("[^a-zA-Z0-9 ]","",title)
  doc=nlp(cleaned)
  f_l=[]
  for token in doc:
    if token.is_stop:
      continue
    f_l.append(token.lemma_)
  return " ".join(f_l)

In [119]:
movies["ori_title"]=movies["title"].copy()
movies["ori_title"]

,ori_title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
...,...
62418,We (2018)
62419,Window of the Soul (2001)
62420,Bad Poems (2018)
62421,A Girl Thing (2001)


In [120]:
movies["title"]=movies["title"].apply(clean_title)
movies["title"]

,title
0,Toy story 1995
1,Jumanji 1995
2,Grumpier Old Men 1995
3,wait Exhale 1995
4,Father Bride II 1995
...,...
62418,2018
62419,window Soul 2001
62420,Bad Poems 2018
62421,Girl thing 2001


In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
v=TfidfVectorizer(ngram_range=(1,2))
tfidf=v.fit_transform(movies["title"])

In [122]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
  title=clean_title(title)
  query_vec=v.transform([title])
  similarity=cosine_similarity(query_vec,tfidf).flatten()
  index=np.argpartition(similarity,-5)[-5:]
  result=movies.iloc[index]
  return result

In [123]:
search("toy story")

,movieId,title,genres,ori_title
20497,106022,Toy Story Terror 2013,Animation|Children|Comedy,Toy Story of Terror (2013)
14813,78499,Toy Story 3 2010,Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 (2010)
59767,201588,Toy Story 4 2019,Adventure|Animation|Children|Comedy,Toy Story 4 (2019)
3021,3114,Toy Story 2 1999,Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 (1999)
0,1,Toy story 1995,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)


In [124]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


In [125]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [126]:
movieid=296

In [127]:
similar_users=ratings[(ratings.movieId==movieid)&(ratings.rating>=4)]["userId"].unique()
similar_users

array([     1,      3,      4, ..., 162533, 162534, 162536])

In [128]:
similar_movies=ratings[(ratings["userId"].isin(similar_users))&(ratings.rating>=4)]["movieId"]
similar_movies

,movieId
0,296
2,307
3,665
5,1088
8,1237
...,...
24999518,148626
24999519,148685
24999520,164179
24999521,202429


In [129]:
movie_recs=similar_movies.value_counts()/len(similar_users)
movie_recs

,count
movieId,
296,1.000000
318,0.630303
593,0.585443
356,0.498382
50,0.497469
...,...
177503,0.000016
188693,0.000016
194698,0.000016


In [130]:
movie_recs=movie_recs[movie_recs>.10]
movie_recs

,count
movieId,
296,1.000000
318,0.630303
593,0.585443
356,0.498382
50,0.497469
...,...
1954,0.102346
500,0.101464
367,0.101464


In [131]:
all_users=ratings[(ratings["movieId"].isin(movie_recs.index))&(ratings.rating>=4)]
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
11,1,1653,4.0,1147868097
19,1,2692,5.0,1147869100
23,1,3949,5.0,1147868678
29,1,4973,4.5,1147869080
...,...,...,...,...
25000078,162541,7153,5.0,1240952613
25000081,162541,7361,4.5,1240953484
25000084,162541,8961,4.0,1240953338
25000087,162541,33794,4.0,1240951792


In [132]:
all_u_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())
all_u_recs

,count
movieId,
318,0.441687
296,0.390963
356,0.368782
593,0.363107
2571,0.349158
...,...
4979,0.060039
6502,0.059970
555,0.059726


In [133]:
rec_per=pd.concat([movie_recs,all_u_recs],axis=1)
rec_per.columns=["similar","all"]
rec_per

,similar,all
movieId,,
296,1.000000,0.390963
318,0.630303,0.441687
593,0.585443,0.363107
356,0.498382,0.368782
50,0.497469,0.285931
...,...,...
1954,0.102346,0.062632
500,0.101464,0.087136
367,0.101464,0.070098


In [134]:
rec_per["score"]=rec_per["similar"]/rec_per["all"]
rec_per=rec_per.sort_values("score",ascending=False)
rec_per.head(10).merge(movies,left_index=True,right_on="movieId")

,similar,all,score,movieId,title,genres,ori_title
292,1.000000,0.390963,2.557790,296,Pulp Fiction 1994,Comedy|Crime|Drama|Thriller,Pulp Fiction (1994)
1062,0.358450,0.164306,2.181595,1089,Reservoir Dogs 1992,Crime|Mystery|Thriller,Reservoir Dogs (1992)
285,0.129102,0.060064,2.149408,288,Natural Born Killers 1994,Action|Crime|Thriller,Natural Born Killers (1994)
549,0.127163,0.059726,2.129122,555,true Romance 1993,Crime|Thriller,True Romance (1993)
4158,0.127403,0.062626,2.034361,4262,scarface 1983,Action|Crime|Drama,Scarface (1983)
9778,0.168915,0.084718,1.993847,32587,Sin City 2005,Action|Crime|Film-Noir|Mystery|Thriller,Sin City (2005)
7299,0.224269,0.112486,1.993760,7438,kill Bill Vol 2 2004,Action|Drama|Thriller,Kill Bill: Vol. 2 (2004)
15,0.143937,0.072416,1.987647,16,casino 1995,Crime|Drama,Casino (1995)
762,0.257947,0.129811,1.987088,778,trainspotte 1996,Comedy|Crime|Drama,Trainspotting (1996)
2451,0.174458,0.088032,1.981768,2542,lock Stock Smoking Barrels 1998,Comedy|Crime|Thriller,"Lock, Stock & Two Smoking Barrels (1998)"


In [135]:
def recommendations(movieid):
  similar_users=ratings[(ratings.movieId==movieid)&(ratings.rating>=4)]["userId"].unique()
  similar_movies=ratings[(ratings["userId"].isin(similar_users))&(ratings.rating>=4)]["movieId"]
  movie_recs=similar_movies.value_counts()/len(similar_users)
  movie_recs=movie_recs[movie_recs>.10]
  all_users=ratings[(ratings["movieId"].isin(movie_recs.index))&(ratings.rating>=4)]
  all_u_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())
  rec_per=pd.concat([movie_recs,all_u_recs],axis=1)
  rec_per.columns=["similar","all"]
  rec_per["score"]=rec_per["similar"]/rec_per["all"]
  rec_per=rec_per.sort_values("score",ascending=False)
  return rec_per.head(10).merge(movies,left_index=True,right_on="movieId")[["score","ori_title","genres"]]



In [136]:
def get_rec(movie_name):
  result=search(movie_name)
  movieid=result.iloc[0]["movieId"]
  return recommendations(movieid)

In [137]:
get_rec("toy story")

,score,ori_title,genres
20497,1620.750000,Toy Story of Terror (2013),Animation|Children|Comedy
24062,1170.541667,Toy Story Toons: Partysaurus Rex (2012),Animation|Children|Comedy
22634,891.412500,Toy Story Toons: Small Fry (2011),Adventure|Animation|Children|Comedy|Fantasy
22633,784.233871,Toy Story Toons: Hawaiian Vacation (2011),Adventure|Animation|Children|Comedy|Fantasy
24064,614.767241,Toy Story That Time Forgot (2014),Animation|Children
18227,213.725275,Boundin' (2003),Animation|Children
16158,212.825758,Winnie the Pooh and Tigger Too (1974),Animation|Children
20494,185.582061,"Blue Umbrella, The (2013)",Animation
17440,162.075000,Arthur Christmas (2011),Animation|Children|Comedy|Drama
21284,161.068323,Mr. Peabody & Sherman (2014),Adventure|Animation|Comedy


In [138]:
# pip install ipywidgets